### Что нужно сделать?
    (+) переделать получение описаний с запроса к api на поиск по данной html-странице
    (+) обработать все базы и выделить нужные вакансии, перенести в новую базу
    (+) выделить из каждой вакансии список требований
    (+) перенести сюда то, что скинул Саня
    
    (-) сделать окончательную удобную! базу (id, работодатель, дата публикации, город, требования) 
    (-) попробовать сделать выделение одного требования через bs4
    (-) проверить все на предмет неправильной работы (все этапы)
    (-) ЗАПИХНУТЬ УЖЕ ВСЕ НАКОНЕЦ В SCIKIT.LEARN

### Входные данные
    > сделанная вручную база, в которой для каждой вакансии (из разных компаний) есть ее id и полный html-текст


    БУДЕМ СЧИТАТЬ, ЧТО НИЧЕГО НЕ ПОТЕРЯЛИ НА ЭТАПЕ ОБРАБОТКИ ИСХОДНЫХ БАЗ

In [2]:
import sqlite3
import json
import requests
from bs4 import BeautifulSoup

In [3]:
import re
import string
import collections

In [4]:
final = 'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\final.db'

conn = sqlite3.connect(final)
cursor = conn.cursor()

cursor.execute('SELECT * from vacancy')
final_db = cursor.fetchall()
conn.close()

### Небольшой предварительный анализ всех заголовков

#### Сбор всех заголовков

In [5]:
headers = []
headers_d = {} # для контроля качества
punct = '«»•∙' + string.punctuation 
table = str.maketrans({key: None for key in punct}) # таблица перевода знаков препинания в None-объект

for job in final_db:
    new_headers = [y for y in [x.text.translate(table).strip().lower() for x in BeautifulSoup(job[1]).findAll('strong')]
                   if len(y) >= 4]
    headers_d[job[0]] = new_headers
    headers += new_headers

In [6]:
headers[:5]

['обязанности', 'требования', 'условия', 'требования', 'условия работы']

In [7]:
print(len(headers))      # неуникальных
print(len(set(headers))) # уникальных

4810
779


In [97]:
# забыл, зачем делал (удаляет заголовки, в которых нет кириллического текста)

#for head in headers:
#    if set('abcdefghijklmnopqrstuvwxyz') & set(head) != set():
#        print(head)

#### Попытка кластеризовать заголовки

In [176]:
test_headers = list(set(headers))

#### Выделение нужных заголовков
    ручная оценка заголовков, подборка ключевых слов, по которым будут отбираться нужные заголовки
    
    
    БУДЕМ СЧИТАТЬ, ЧТО ВЗЯЛИ ВСЕ, ЧТО НУЖНО И НЕ ВЗЯЛИ ТО, ЧТО НЕ НУЖНО, НО ЭТО НЕ ТОЧНО))))))))0)

In [8]:
counter_d = sorted(collections.Counter(headers).items(), key = lambda x: x[1], reverse=True)
counter_d[:10]

[('требования', 569),
 ('обязанности', 490),
 ('мы предлагаем', 422),
 ('условия', 331),
 ('желательно', 153),
 ('будет плюсом', 112),
 ('для чего вы нам нужны', 101),
 ('задачи', 92),
 ('мы предлагаем кучу всего', 85),
 ('что вам необходимо для этого', 82)]

In [ ]:
stopwords = []

In [39]:
keywords = ['требования', 'желательно', 'плюсом', 'необходимо', 'ожидаем', 'requirements', 'обязательно', 
            'приветствуется', 'плюсами', 'знание', 'ожидания', 'компетенции', 'приветствуются', 
            'backend', 'технологии', 'требуется', 'если', 'бонусы', 'ждём', 'важно', 'плюсы', 'плюс', 
            'навыки', 'будет', 'ищем', 'уровень', 'вас', 'дополнительно', 'желательны', 'техническое',
            'ждем', 'стек', 'ожиданем', 'умеете', 'приветствуем', 'навыки', 'нужно', 'стеке', 'потребуется', 
            'преимуществом', 'нужен', 'ожиданием', 'пригодится', 'expect', 'fullstack', 'качества', 
            'looking', 'хотелось', 'требовани', 'обязательн', 'must', 'skills', 'мастхэв', 'required',
            'ходим', 'обязателен', 'тебя', 'преимущества', 'знаниеумения', 'пожелания', 'стэк', 'компетенция', 
            'желательные', 'преимущество', 'хотим', 'знания', 'желание', 'хотели', 'have', 'применяется', 'технологий',
            'видим', 'обязательные', 'разработки']

In [ ]:
questionable_kwords = ['нужно', 'опыт', 'experience', '']

In [ ]:
unaccaptable_phrases = ['что тебя ждет на проекте', 'что не обязательно для работы у нас', 'что не нужно будет делать',
                        'что будет делать стажер'
                       ]

In [10]:
def check(word_list):
    global keywords
    return set(keywords) & set(word_list) != set()

In [40]:
filtered_head = []
for head in set(headers):
    word_list = head.split()
    if set(keywords) & set(word_list) != set():
        filtered_head.append(head)

In [24]:
filtered_head = []
for job in headers_d:
    for head in headers_d[job]:
        word_list = head.split()
        if set(keywords) & set(word_list) != set():
            filtered_head.append((job, head))

In [54]:
set(filtered_head)

{'additional requirements',
 'backend разработчика следующий',
 'bтехнические компетенции',
 'desired skills and experience',
 'eщё желательно',
 'fullstack',
 'fullstack developer',
 'functional knowledge and skills',
 'key requirements',
 'knowledge skills and abilities',
 'mandatory skills',
 'must have',
 'must have soft skills',
 'nice to have',
 'other requirements',
 'personal skills',
 'professional knowledge and skills',
 'qualification requirements',
 'required',
 'requirements',
 'spring hibernate gradle jms jaxws git jenkins uml junit знание основ рсубд',
 'what are we looking for',
 'what we expect from you',
 'а еще желательно',
 'а также у вас есть опыт',
 'английский язык не ниже уровня upperintermediate навыки общения в письменной форме лично и по телефону',
 'большим плюсом будет',
 'большим плюсом будут',
 'большим преимуществом для тебя будет',
 'бонусы',
 'бонусы и скидки',
 'будет',
 'будет большим плюсом',
 'будет здорово если вы',
 'будет плюсом',
 'будет плюсом

#### Формирование словаря 'заголовок - содержание'
    129/1215 вакансий не были обработаны должным образом, еще бог знает сколько требований нужно разделять вручную.

In [56]:
# удаление html-кода из текста
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return ' '.join(cleantext.split())

In [24]:
filtered_head = []
for job in headers_d:
    for head in headers_d[job]:
        word_list = head.split()
        if set(keywords) & set(word_list) != set():
            filtered_head.append((job, head))

In [78]:
all_paragraphs = {}
errors = []
punct = '«•»∙!"#$%&\'()*+-./:;<=>?@[\\]^_`{|}~'
table = str.maketrans({key: None for key in punct})

for job in final_db:
    text = job[1]
    if '<strong>' in text:
        paragraphs = [x.split('</strong>') for x in text.split('<strong>') if '</strong>' in x]
        all_paragraphs[job[0]] = {x[0].translate(table).strip().lower(): x[1] for x in paragraphs}
    else:
        errors.append(job[0])

In [79]:
print(len(all_paragraphs))
print(len(set(errors)))

1167
48


In [80]:
requirements = {}
requirements_l = []
for id_ in all_paragraphs: 
    for paragraph in all_paragraphs[id_]:
        word_list = paragraph.split()
        if set(word_list) & set(keywords) != set():              # а что если несколько заголовков подойдут?
            new_reqs = all_paragraphs[id_][paragraph].strip()
            if cleanhtml(new_reqs) != '':
                requirements[id_] = new_reqs
                requirements_l.append(new_reqs)
            else:
                errors.append(id_)

In [81]:
requirements_l[1]

'</p> <ul> <li>Опыт web разработки от 3 лет (HTML, HTML5, CSS, JavaScript, препроцессинг CSS)</li> <li>Опыт разработки на библиотеках и фреймворках (AngularJS, ReactJS)</li> <li>Активное использование инструментов контроля версий (Git, CVS)</li> <li>Умение работать с базами данных и языками запросов (SQL)</li> <li>Опыт работы с серверными технологиями (Node.js)</li> <li>Взаимодействие с бекендом (SOAP/REST)</li> </ul> <p> </p> <p>'

In [61]:
errors = errors + list(set(all_paragraphs.keys()).difference(set(requirements.keys()))) # добавил потерянные вакансии

#### а не 129 (??????)

In [82]:
len(set([x[0] for x in final_db]).difference(set(requirements.keys()))) # те вакансии, для которых не получен список требований

128

#### Разделение на мысли (от Сани)
    все равно берется куча требований, не имеющих отношения к тех. навыкам:
     1. как будто какие-то рандомные заголовки
     2. личные качества (soft skills)
     3. какая-то хрень

In [83]:
work_list = []
for i in requirements_l:
    work_list.append(i.split('>'))

In [84]:
new_work_list = []
for i in work_list:
    ap_list = []
    for j in i:
        ap_list.append(j.split('<'))
    new_work_list.append(ap_list)

In [85]:
flatten_list = []
for i in new_work_list:
    for j in i:
        for k in j:
            flatten_list.append(k)

In [86]:
trash_list = ['ul', 'li', 'p', 'b', '/ul', '/li', 
              '/p', '/b', 'em','/em','div','/div',
              'br','/br', 'ol', '/ol', '/span', 'u', '/u',
             ' ', '', 'br/', 'span', ':', 'br /']

In [87]:
requirements_final = []
for i in flatten_list:
    if i not in trash_list:
        requirements_final.append(i)

In [90]:
requirements_final = [x.translate(table).strip().lower() for x in requirements_final]

In [91]:
requirements_final[:5]

['законченное высшее образование по направлению информационные технологии, возможно последний курс',
 'желателен опыт работы в сфере ит',
 'экспертное знание ms windows 7 и ms office 2010 и выше',
 'опыт поддержки пк, системного по, в тч навыки удаленного администрирования пк',
 'знание технического устройства пк и его аппаратных составляющих']

In [177]:
sorted(set(dict(collections.Counter(requirements_final)).items()), key=lambda x: x[1], reverse=True)

[('высшее техническое образование', 111),
 ('технический английский', 73),
 ('высшее образование', 58),
 ('условия работы', 41),
 ('умение работать в команде', 41),
 ('интерес к исследовательской деятельности', 32),
 ('технический английский – свободное чтение документации', 32),
 ('желательно', 31),
 ('знание классических алгоритмов и структур данных', 27),
 ('умение разбираться в чужом коде', 24),
 ('опыт работы с git', 24),
 ('будет плюсом', 24),
 ('опыт работы в соответствии с методологиями agile', 24),
 ('ответственность, исполнительность, инициативность, внимание к деталям', 20),
 ('знание методологических подходов проектирования программного обеспечения',
  19),
 ('образование  высшее техническое образование, предпочтительно в области ит',
  18),
 ('умение самостоятельно и полностью доводить до конца поставленные задачи',
  17),
 ('опыт разработки многопоточных приложений', 17),
 ('знание алгоритмов и структур данных', 15),
 ('знание sql', 15),
 ('знание методов обеспечения безо

In [96]:
[x.split(',') for x in requirements_final if ',' in x]

[['законченное высшее образование по направлению информационные технологии',
  ' возможно последний курс'],
 ['опыт поддержки пк',
  ' системного по',
  ' в тч навыки удаленного администрирования пк'],
 ['стрессоустойчивость', ' ответственность', ' коммуникабельность'],
 ['опыт web разработки от 3 лет html',
  ' html5',
  ' css',
  ' javascript',
  ' препроцессинг css'],
 ['опыт разработки на библиотеках и фреймворках angularjs', ' reactjs'],
 ['активное использование инструментов контроля версий git', ' cvs'],
 ['опыт web разработки от 3 лет html',
  ' html5',
  ' css',
  ' javascript',
  ' препроцессинг css'],
 ['опыт разработки на библиотеках и фреймворках angularjs', ' reactjs'],
 ['активное использование инструментов контроля версий git', ' cvs'],
 ['знание itsm', ' itil'],
 ['опыт работы с jira', ' confluence'],
 ['желательно опыт использования и построения отчётов в grafana',
  ' zabbix',
  ''],
 ['желательно опыт использования технологий devops и версионного контроля jenkins',
